# Upload Dataset

In [1]:
from google.colab import files
uploaded = files.upload()

Saving allHouses.csv to allHouses.csv


In [2]:
import pandas as pd
import io

house_df = pd.read_csv(io.StringIO(uploaded['allHouses.csv'].decode('utf-8')))
print(house_df.columns.to_list())

['HouseNum', 'TimeSin', 'TimeCos', 'DayNumSin', 'DayNumCos', 'MonthSin', 'MonthCos', 'RealTemp', 'ApparTemp', 'Humid', 'Wmo_0-9', 'Wmo_10-19', 'Wmo_20-29', 'Wmo_30-39', 'Wmo_40-49', 'Wmo_50-59', 'Wmo_60-69', 'Wmo_70-79', 'Wmo_80-89', 'Wmo_90-99', 'YearBuilt__under_1899', 'YearBuilt_1900-1909', 'YearBuilt_1910-1919', 'YearBuilt_1920-1929', 'YearBuilt_1930-1939', 'YearBuilt_1940-1949', 'YearBuilt_1950-1959', 'YearBuilt_1960-1969', 'YearBuilt_1970-1979', 'YearBuilt_1980-1989', 'YearBuilt_1990-1999', 'YearBuilt_2000-2009', 'YearBuilt_2010-2019', 'YearBuilt_2020-2029', 'Type_BUNGALOW', 'Type_COTTAGE', 'Type_DETACHED', 'Type_END OF TERRACE', 'Type_FLAT', 'Type_SEMI-DETACHED', 'Type_MID-TERRACE', 'Type_STUDENT HALLS', 'Type_FACTORY', 'Type_OFFICE', 'Type_UNIVERSITY', 'NumRooms', 'NumOccupants', 'Total', 'AlwaysOn', 'Intermit', 'HVAC']


In [2]:
import pandas as pd

file_path = '/content/allHouses.csv'

# Read the CSV file into a DataFrame
house_df = pd.read_csv(file_path)
print(house_df.columns.to_list())

['HouseNum', 'TimeSin', 'TimeCos', 'DayNumSin', 'DayNumCos', 'MonthSin', 'MonthCos', 'RealTemp', 'ApparTemp', 'Humid', 'Wmo_0-9', 'Wmo_10-19', 'Wmo_20-29', 'Wmo_30-39', 'Wmo_40-49', 'Wmo_50-59', 'Wmo_60-69', 'Wmo_70-79', 'Wmo_80-89', 'Wmo_90-99', 'YearBuilt__under_1899', 'YearBuilt_1900-1909', 'YearBuilt_1910-1919', 'YearBuilt_1920-1929', 'YearBuilt_1930-1939', 'YearBuilt_1940-1949', 'YearBuilt_1950-1959', 'YearBuilt_1960-1969', 'YearBuilt_1970-1979', 'YearBuilt_1980-1989', 'YearBuilt_1990-1999', 'YearBuilt_2000-2009', 'YearBuilt_2010-2019', 'YearBuilt_2020-2029', 'Type_BUNGALOW', 'Type_COTTAGE', 'Type_DETACHED', 'Type_END OF TERRACE', 'Type_FLAT', 'Type_SEMI-DETACHED', 'Type_MID-TERRACE', 'Type_STUDENT HALLS', 'Type_FACTORY', 'Type_OFFICE', 'Type_UNIVERSITY', 'NumRooms', 'NumOccupants', 'Total', 'AlwaysOn', 'Intermit', 'HVAC']


In [ ]:
import pandas as pd
import io

# Read in the csv
houses_dataset = r'C:'
house_df = pd.read_csv(houses_dataset)


# Setting up model

In [3]:
import numpy as np
import pandas as pd

# print(house_df.head())
print("house_df---->>>>",house_df.columns.to_list())
# selected_features = ['AlwaysOn', 'Intermit', 'HVAC']

X = house_df.iloc[:, :-3] # all the features for the training
Y = house_df.iloc[:, -3:] # Last 3 are ['AlwaysOn', 'Intermit', 'HVAC']

print("X.head----->>>>>", X.columns.to_list())
print("Y.head----->>>>>",Y.columns.to_list())

house_df---->>>> ['HouseNum', 'TimeSin', 'TimeCos', 'DayNumSin', 'DayNumCos', 'MonthSin', 'MonthCos', 'RealTemp', 'ApparTemp', 'Humid', 'Wmo_0-9', 'Wmo_10-19', 'Wmo_20-29', 'Wmo_30-39', 'Wmo_40-49', 'Wmo_50-59', 'Wmo_60-69', 'Wmo_70-79', 'Wmo_80-89', 'Wmo_90-99', 'YearBuilt__under_1899', 'YearBuilt_1900-1909', 'YearBuilt_1910-1919', 'YearBuilt_1920-1929', 'YearBuilt_1930-1939', 'YearBuilt_1940-1949', 'YearBuilt_1950-1959', 'YearBuilt_1960-1969', 'YearBuilt_1970-1979', 'YearBuilt_1980-1989', 'YearBuilt_1990-1999', 'YearBuilt_2000-2009', 'YearBuilt_2010-2019', 'YearBuilt_2020-2029', 'Type_BUNGALOW', 'Type_COTTAGE', 'Type_DETACHED', 'Type_END OF TERRACE', 'Type_FLAT', 'Type_SEMI-DETACHED', 'Type_MID-TERRACE', 'Type_STUDENT HALLS', 'Type_FACTORY', 'Type_OFFICE', 'Type_UNIVERSITY', 'NumRooms', 'NumOccupants', 'Total', 'AlwaysOn', 'Intermit', 'HVAC']
X.head----->>>>> ['HouseNum', 'TimeSin', 'TimeCos', 'DayNumSin', 'DayNumCos', 'MonthSin', 'MonthCos', 'RealTemp', 'ApparTemp', 'Humid', 'Wmo_0-

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import PowerTransformer
import numpy as np
import pandas as pd

# Assuming X is your feature matrix and Y is the target matrix with 3 columns
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

total_energy_test = pd.DataFrame(X_test['Total'])
print("Original total energy test data, X_test:", total_energy_test.sort_index())

# Define the inverse Yeo-Johnson function
def inverse_yeo_johnson(y, lambda_):
    if lambda_ == 0:
        return np.exp(y) - 1
    else:
        if y >= 0:
            return (y * lambda_ + 1) ** (1 / lambda_) - 1
        else:
            return -((1 - y * (2 - lambda_)) ** (1 / (2 - lambda_)) - 1)


lambda_value = -0.06419593996677918
total_energy_inverseYeoJohnson = total_energy_test.apply(lambda row: inverse_yeo_johnson(row['Total'], lambda_value), axis=1)
total_energy_inverseYeoJohnson_df = pd.DataFrame(total_energy_inverseYeoJohnson, columns=['Total'])
print("total_energy_inverseYeoJohnson_df\n",total_energy_inverseYeoJohnson_df.sort_index())
print("total_energy_inverseYeoJohnson_df.shape", total_energy_inverseYeoJohnson_df.shape)

print("total_energy_inverseYeoJohnson :\n ",total_energy_inverseYeoJohnson.sort_index())
print("y_test____\n",y_test.sort_index())
print("y_test.shape", y_test.shape)


# Convert DataFrames to numpy arrays
total_energy_inverseYeoJohnson_array = total_energy_inverseYeoJohnson_df.values
y_test_array = y_test.values

y_test_actuals = y_test_array * total_energy_inverseYeoJohnson_array
y_test_actuals_df = pd.DataFrame(y_test_actuals, index=y_test.index, columns=y_test.columns)
print("result: \n",y_test_actuals_df.sort_index())


Original total energy test data, X_test:            Total
4       4.791903
6       3.172128
12      4.472217
22      5.796822
24      5.150908
...          ...
220455  5.946285
220456  5.946285
220458  5.946285
220462  5.946285
220466  5.946285

[44094 rows x 1 columns]
total_energy_inverseYeoJohnson_df
            Total
4        305.909
6         33.708
12       193.716
22      1407.225
24       519.030
...          ...
220455  1789.000
220456  1789.000
220458  1789.000
220462  1789.000
220466  1789.000

[44094 rows x 1 columns]
total_energy_inverseYeoJohnson_df.shape (44094, 1)
total_energy_inverseYeoJohnson :
  4          305.909
6           33.708
12         193.716
22        1407.225
24         519.030
            ...   
220455    1789.000
220456    1789.000
220458    1789.000
220462    1789.000
220466    1789.000
Length: 44094, dtype: float64
y_test____
         AlwaysOn  Intermit      HVAC
4       0.296886  0.100860  0.602254
6       0.055180  0.220333  0.724487
12      0.187666

# Performance Analytics

Definitions:<br>
**MSE**:<br>
-> Definition: MSE measures the average of
the squared differences between actual and predicted values. It is calculated by taking the average of the square of the errors (difference between actual and predicted values). <br>
-> Usage: MSE is useful for evaluating the variance of the model's errors. Lower MSE values indicate better model performance
<br>
**RMSE**:<br>
-> Definition: RMSE is the square root of the mean squared error. It measures the standard deviation of the residuals (errors).<br>
-> Usage: RMSE is more interpretable than MSE because it is in the same units as the target variable. It provides insight into how close the model's predictions are to the actual values.
<br>
**MAE**:<br>
-> Definition: MAE measures the average of the absolute differences between actual and predicted values. It is calculated by taking the mean of the absolute differences between actual and predicted values.<br>
-> Usage: MAE is useful for measuring the average magnitude of errors, without considering their direction. Lower MAE values indicate better model performance.
<br>
**MAPE**:<br>
-> Definition: MAPE measures the average of the absolute percentage differences between actual and predicted values. It expresses errors as percentages of actual values.<br>
-> Usage: MAPE provides insight into the relative size of errors. It is useful for comparing model performance across different scales.
<br>
**R^2**:<br>
-> Definition: R^2 measures the proportion of the variance in the target variable that is explained by the model. It ranges from 0 to 1, where 1 indicates a perfect fit.<br>
-> Usage: R^2 provides a measure of how well the model explains the variability of the target variable. Higher values indicate better model performance.

In [31]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import numpy as np
def calculate_performance_metrics(y_test_actuals, y_pred_actuals_model):
    # Calculate performance metrics
    metrics = {}

    metrics['mse'] = mean_squared_error(y_test_actuals, y_pred_actuals_model, multioutput='raw_values')
    metrics['rmse'] = np.sqrt(metrics['mse'])
    metrics['mae'] = mean_absolute_error(y_test_actuals, y_pred_actuals_model, multioutput='raw_values')
    metrics['mape'] = mean_absolute_percentage_error(y_test_actuals, y_pred_actuals_model, multioutput='raw_values')
    metrics['r2'] = r2_score(y_test_actuals, y_pred_actuals_model, multioutput='raw_values')


    # Check if any of the predicted values are negative
    negative_values = y_pred_actuals_model < 0
    num_negative_values = np.sum(negative_values)
    if num_negative_values > 0:
        print(f"Warning!!: {num_negative_values} predicted values are negative")

    return metrics

def print_performance_metrics(metrics):
    print("Y.head: ",Y.columns.to_list())
    print("MSE per output:", metrics['mse'])
    print("RMSE per output:", metrics['rmse'])
    print("MAE per output:", metrics['mae'])
    print("MAPE per output:", metrics['mape'])
    print("R^2 per output:", metrics['r2'])



# Linear Regression

In [25]:
import pandas as pd

# Print the first five rows of y_train
print(y_train.head())

# Check for missing values in y_train
print(y_train.isnull().sum())

# Initialize and fit the model
linear_model = MultiOutputRegressor(ElasticNet(random_state=42, positive=True)).fit(X_train, y_train)

# Predicting the percentages
y_pred_fracs_linear = linear_model.predict(X_test)
print("y_pred_fracs_linear", y_pred_fracs_linear)


# Converting back to energy values
y_pred_actuals_linear = y_pred_fracs_linear * total_energy_inverseYeoJohnson_array
result_df = pd.DataFrame(y_pred_actuals_linear)
print("result: ",result_df.sort_index())
# performance analysis
metrics = calculate_performance_metrics(y_test_actuals, y_pred_actuals_linear)
print_performance_metrics(metrics)

        AlwaysOn  Intermit      HVAC
162687  0.080576  0.089216  0.830208
22801   0.190761  0.098911  0.710328
203564  0.396688  0.030601  0.572711
140524  0.000000  0.166675  0.833325
129027  0.126301  0.595352  0.278347
AlwaysOn    0
Intermit    0
HVAC        0
dtype: int64
y_pred_fracs_linear [[0.18243149 0.18373948 0.63382902]
 [0.18243149 0.18373948 0.63382902]
 [0.18243149 0.18373948 0.63382902]
 ...
 [0.18243149 0.18373948 0.63382902]
 [0.18243149 0.18373948 0.63382902]
 [0.18243149 0.18373948 0.63382902]]
result:                  0           1           2
0      113.542990  114.357066  394.486944
1      131.349581  132.291326  456.353094
2      122.128398  123.004030  424.315572
3      121.128127  121.996586  420.840287
4       15.342489   15.452491   53.305021
...           ...         ...         ...
44089   41.276402   41.572344  143.408253
44090   25.677233   25.861332   89.211435
44091  158.865358  160.004384  551.952258
44092  109.892353  110.680255  381.803392
44093   43

# Graphs

In [ ]:
### residuals against predicted values###

import matplotlib.pyplot as plt


# Calculate residuals
residuals_linear = y_test_actuals - y_pred_actuals_linear


# Plot residuals against predicted values
for column in residuals_linear.columns:

    # Extract the predicted values and residuals for the current target variable
    current_predicted_values = y_pred_actuals_linear[:, residuals_linear.columns.get_loc(column)]
    current_residuals = residuals_linear[column]

    # Create a scatter plot of residuals vs. predicted values
    plt.figure(figsize=(8, 6))
    plt.scatter(current_predicted_values, current_residuals)
    plt.axhline(0, color='red', linestyle='--')
    plt.xlabel('Predicted Values')
    plt.ylabel('Residuals')
    plt.title(f'Residuals vs. Predicted Values (Linear Model) - {column}')
    plt.show()


In [ ]:
###QQ plot###

import matplotlib.pyplot as plt
import scipy.stats as stats

# Calculate residuals
residuals_linear = y_test_actuals - y_pred_actuals_linear

# QQ plot of residuals
# Loop through each column in the residuals
for column in residuals_linear.columns:
    # Extract the residuals for the current column
    current_residuals = residuals_linear[column]

    # Create a QQ plot for the current residuals
    plt.figure(figsize=(8, 6))
    stats.probplot(current_residuals, dist="norm", plot=plt)
    plt.title(f'QQ Plot of Residuals (Linear Model) - {column}')
    plt.show()

In [ ]:
# from sklearn.inspection import plot_partial_dependence
# import matplotlib.pyplot as plt

# # Features for which you want to plot PDPs
# features_to_plot = ['Feature1', 'Feature2', 'Feature3']  # Replace with the features you want to visualize

# # Output variables to visualize (index corresponds to the order of columns in y_train)
# target_indices = [0, 1, 2]  # HVAC_frac, AlwaysOn_frac, Intermit_frac

# # Create PDPs
# fig, ax = plt.subplots(figsize=(10, 8))
# plot_partial_dependence(
#     estimator=linear_model,
#     X=X_train,
#     features=features_to_plot,
#     target=target_indices,
#     ax=ax,
#     grid_resolution=50  # Number of points to use in grid
# )

# # Set titles for the plot
# ax[0].set_title('HVAC_frac')
# ax[1].set_title('AlwaysOn_frac')
# ax[2].set_title('Intermit_frac')

# # Show plot
# plt.show()


In [ ]:
# ###SHAP (Shapley Additive Explanations)###

# import shap
# import matplotlib.pyplot as plt


# # Initialize the SHAP explainer for your model
# explainer = shap.Explainer(linear_model, X_train)

# # Compute SHAP values for the test set
# shap_values = explainer.shap_values(X_test)

# # Create SHAP summary plot for each target variable
# for i, target in enumerate(['HVAC', 'AlwaysOn', 'Intermit']):
#     # Use shap.summary_plot for each target variable
#     shap.summary_plot(shap_values[i], X_test, feature_names=selected_features, show=False)
#     plt.title(f'SHAP Summary Plot for {target}')
#     plt.show()

# Random Forest Regressor

In [36]:
from sklearn.ensemble import RandomForestRegressor

# Initialize and fit the model
rf_model = MultiOutputRegressor(RandomForestRegressor(
    verbose=1,
    n_jobs=-1,
    n_estimators=100, #try 50 # was 100
    # max_depth = 10, #default is none
    # max_features=0.5, # Fewer features per split, less memory
    random_state=42
))

rf_model.fit(X_train, y_train)

# Predict the fractions
y_pred_fracs_rf = rf_model.predict(X_test)


##########MODEL 2 batching#########################
###### I dont think this works#######


# from sklearn.ensemble import RandomForestRegressor
# batch_size = 16  # make lower if crashing
# #512 , 256, 64, 32, 16,8
# forest_model = RandomForestRegressor(
#     verbose=1,
#     n_jobs=-1,
#     n_estimators=16, #try 50 # was 100
#     # max_depth = 10, #default is none
#     # max_features=0.5, # Fewer features per split, less memory
#     random_state=42
# )

# start_idx = 0
# while start_idx < len(X_train):
#     # Get the end index for the current batch
#     end_idx = min(start_idx + batch_size, len(X_train))

#     # Get the current batch of data
#     X_batch = X_train[start_idx:end_idx]
#     y_batch = y_train[start_idx:end_idx]

#     # Fit the model on the current batch
#     forest_model.fit(X_batch, y_batch)

#     # Move to the next batch
#     start_idx = end_idx


# y_pred_fracs_forest = []
# start_idx = 0
# while start_idx < len(X_test):
#     # Get the end index for the current batch
#     end_idx = min(start_idx + batch_size, len(X_test))

#     # Get the current batch of data
#     X_test_batch = X_test[start_idx:end_idx]

#     # Predict the fractions for the current batch
#     y_pred_fracs_forest_batch = forest_model.predict(X_test_batch)

#     # Append the predictions to the overall list
#     y_pred_fracs_forest.extend(y_pred_fracs_forest_batch)

#   # Move to the next batch
#     start_idx = end_idx


# Converting back to energy values
y_pred_actuals_rf = y_pred_fracs_rf * total_energy_inverseYeoJohnson_array

# calculate metrics
metrics = calculate_performance_metrics(y_test_actuals, y_pred_actuals_rf)
print_performance_metrics(metrics)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.5min finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.5min finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.5min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.8s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    1.7s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.8s
[Paral

MSE per output: [ 1248.94114297 15374.91660792 15620.75241501]
RMSE per output: [ 35.34036139 123.99563141 124.98300851]
MAE per output: [17.47298516 53.00004516 56.25743013]
MAPE per output: [2.29175142e+15 6.52421968e+15 4.24229230e+12]
R^2 per output: [0.63607953 0.80154806 0.90569194]


[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    1.6s finished


## Random Forest Regressor GridSearchCV

In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
import numpy as np

param_grid = {
    'estimator__n_estimators': [50, 100],
    'estimator__max_depth': [5, 10],
    'estimator__min_samples_split': [2, 5],
    'estimator__min_samples_leaf': [1, 2],
    'estimator__max_features': ['sqrt'],
}

# param_grid = {
#     'estimator__n_estimators': [100, 200, 300],
#     'estimator__max_depth': [10, 20, 30],
#     'estimator__min_samples_split': [5, 10, 15],
#     'estimator__min_samples_leaf': [2, 4, 6],
#     'estimator__max_features': ['sqrt', 'log2'],
# }

rf_model = MultiOutputRegressor(RandomForestRegressor(
    verbose=1,
    n_jobs=-1,
    n_estimators=100, #try 50 # was 100
    # max_depth = 10, #default is none
    # max_features=0.5, # Fewer features per split, less memory
    random_state=42
))
# rf_model = MultiOutputRegressor(RandomForestRegressor(random_state=42))


grid_search_rf = GridSearchCV(
    rf_model,
    param_grid,
    cv=3,
    scoring='neg_mean_squared_error',
    n_jobs=-1)
grid_search_rf.fit(X_train, y_train)
best_rf_model = grid_search_rf.best_estimator_
best_params_ = grid_search_rf.best_params_

print("best_params_ ----> Random Forest:", best_params_)
print("best_rf_model: ", best_rf_model)
y_pred_fracs_best_rf = best_rf_model.predict(X_test)


# Converting back to energy values
y_pred_actuals_rf = y_pred_fracs_rf * total_energy_inverseYeoJohnson_array
# calculate metrics
metrics = calculate_performance_metrics(y_test_actuals, y_pred_actuals_rf)
print_performance_metrics(metrics)


best_params_ ----> Random Forest: {'estimator__max_depth': 5, 'estimator__max_features': 'sqrt', 'estimator__min_samples_leaf': 2, 'estimator__min_samples_split': 2, 'estimator__n_estimators': 100}
best_rf_model:  MultiOutputRegressor(estimator=RandomForestRegressor(max_depth=5,
                                                     max_features='sqrt',
                                                     min_samples_leaf=2,
                                                     random_state=42))
MSE per output (best RF): [ 1136.66330419 50173.44688656 49124.72627347]
RMSE per output (best RF): [ 33.71443762 223.99430101 221.64098509]
MAE per output (best RF): [ 21.52670528 125.51942511 125.59487467]
MAPE per output (best RF): [7.85907375e+01 1.52998168e+17 2.99874283e+01]
R^2 per output (best RF): [0.0095653  0.72281203 0.73964588]


## Random Forest Regressor RandomizedSearchCV

In [35]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import randint, uniform

param_dist_rf = {
    'estimator__n_estimators': randint(50, 200),  # Number of trees in the forest
    'estimator__max_depth': [None, 10, 20, 30, 40],  # Maximum depth of trees
    'estimator__min_samples_split': randint(2, 10),  # Minimum samples for splitting a node
    'estimator__min_samples_leaf': randint(1, 4),  # Minimum samples per leaf
    # 'estimator__max_features': ['auto', 'sqrt', 'log2', 1.0],  # Number of features to consider for splitting
    'estimator__max_features': [1.0],  # Number of features to consider for splitting
}


# Initialize and fit the model
rf_model = MultiOutputRegressor(RandomForestRegressor(
    verbose=1,
    n_jobs=-1,
    n_estimators=100, #try 50 # was 100
    # max_depth = 10, #default is none
    # max_features=0.5, # Fewer features per split, less memory
    random_state=42
))
# rf_model = MultiOutputRegressor(RandomForestRegressor(random_state=42))


randomized_search_rf = RandomizedSearchCV(
    estimator=rf_model,
    param_distributions=param_dist_rf,
    n_iter=10,  # Number of random combinations to try
    scoring='neg_mean_squared_error',  # Metric to optimize
    # n_jobs=-1,
    cv=3,  # Number of cross-validation folds
    random_state=42  # For reproducibility
)

randomized_search_rf.fit(X_train, y_train)
print("best_params_ ---> RandomForestRegressor:", randomized_search_rf.best_params_)
print("best_rf_model---> RandomForestRegressor:", randomized_search_rf.best_rf_model)


# Converting back to energy values
y_pred_actuals_rf = y_pred_fracs_rf * total_energy_inverseYeoJohnson_array

# calculate metrics
metrics = calculate_performance_metrics(y_test_actuals, y_pred_actuals_rf)
print_performance_metrics(metrics)


KeyboardInterrupt: 

# Gradient Boosting

In [11]:
from xgboost import XGBRegressor

boost_model = MultiOutputRegressor(XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)).fit(X_train, y_train)

# Predict the fractions
y_pred_fracs_gb = boost_model.predict(X_test)

# Converting back to energy values
y_pred_fracs_gb = y_pred_fracs_rf * total_energy_inverseYeoJohnson_array
# calculate metrics
metrics = calculate_performance_metrics(y_test_actuals, y_pred_fracs_gb)
print_performance_metrics(metrics)

result:                  0           1           2
0       53.920039  170.678061  379.050319
1       72.909537  220.926222  451.484971
2      110.571738  122.543179  416.299081
3       73.932349  269.415893  385.676777
4        0.261486    8.729433   71.264160
...           ...         ...         ...
44089   48.799087   20.551639  146.825160
44090   35.700358    5.922294   97.266478
44091   33.582263  121.902000  686.724284
44092   56.536256   74.547774  451.013466
44093   29.436073   23.061281  187.198236

[44094 rows x 3 columns]


## Gradient Boosting GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor

# Define the parameter grid
param_grid_gb = {
    'estimator__n_estimators': [50, 100],
    'estimator__max_depth': [3, 6, 9],
    'estimator__learning_rate': [0.1, 0.01],
    'estimator__subsample': [0.8, 1.0],
}

# Initialize the MultiOutputRegressor with XGBRegressor
gb_model = MultiOutputRegressor(XGBRegressor(objective='reg:squarederror', random_state=42))

# Grid search
grid_search_gb = GridSearchCV(
    estimator=gb_model,
    param_grid=param_grid_gb,
    cv=3,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)
grid_search_gb.fit(X_train, y_train)

# Get the best model and its parameters
best_gb_model = grid_search_gb.best_estimator_
best_params_gb = grid_search_gb.best_params_

print("best_params_ ----> Gradient Boosting:", best_params_gb)
print("best_gb_model: ", best_gb_model)

# Predict using the best model
y_pred_fracs_gb = best_gb_model.predict(X_test)

# Converting back to energy values
y_pred_actuals_gb = y_pred_fracs_gb * total_energy_inverseYeoJohnson_array

# calculate metrics
metrics = calculate_performance_metrics(y_test_actuals, y_pred_actuals_gb)
print_performance_metrics(metrics)


## Gradient Boosting RandomizedSearchCV

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor

# Define the parameter distributions
param_dist_gb = {
    'estimator__n_estimators': randint(50, 200),
    'estimator__max_depth': [3, 6, 9, None],
    'estimator__learning_rate': uniform(0.01, 0.1),
    'estimator__subsample': uniform(0.5, 0.5),
}

# Initialize the MultiOutputRegressor with XGBRegressor
gb_model = MultiOutputRegressor(XGBRegressor(objective='reg:squarederror', random_state=42))

# Randomized search
randomized_search_gb = RandomizedSearchCV(
    estimator=gb_model,
    param_distributions=param_dist_gb,
    n_iter=10,
    cv=3,
    scoring='neg_mean_squared_error',
    random_state=42
)
randomized_search_gb.fit(X_train, y_train)

# Get the best model and its parameters
best_gb_model = randomized_search_gb.best_estimator_
best_params_gb = randomized_search_gb.best_params_

print("best_params_ ----> Gradient Boosting:", best_params_gb)
print("best_gb_model: ", best_gb_model)

# Predict using the best model
y_pred_fracs_gb = best_gb_model.predict(X_test)

# Converting back to energy values
y_pred_actuals_gb = y_pred_fracs_gb * total_energy_inverseYeoJohnson_array

# calculate metrics
metrics = calculate_performance_metrics(y_test_actuals, y_pred_actuals_gb)
print_performance_metrics(metrics)
